In [1]:

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python
!pip install koco

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 30.67 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
import urllib.request
import pandas as pd
import random
import koco

In [4]:
from khaiii import KhaiiiApi
api = KhaiiiApi()


In [5]:
'''
import csv

train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

with open('/content/gdrive/My Drive/datas/hate.csv', 'w', newline='') as csvfile:
    fieldnames = ['document', 'lable']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for train in train_dev['train']:
        lable = 1
        if train['hate'] != 'none':
            lable = 0
        writer.writerow({'document':train['comments'],'lable':lable})
'''

"\nimport csv\n\ntrain_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')\n\nwith open('/content/gdrive/My Drive/datas/hate.csv', 'w', newline='') as csvfile:\n    fieldnames = ['document', 'lable']\n    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)\n\n    writer.writeheader()\n    for train in train_dev['train']:\n        lable = 1\n        if train['hate'] != 'none':\n            lable = 0\n        writer.writerow({'document':train['comments'],'lable':lable})\n"

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
#데이터 전처리 후에 남은 문자들이 전부 빈칸일 때 제거하기 위함
def isAll0(x):
    if type(x) == float:
        return x
    elif len(x) == x.count(' '):
        return ''
    else:
        return x


In [303]:
'''
#데이터 가져와서
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
'''
#읽고
all_df = pd.read_csv('/content/gdrive/My Drive/datas/hate_2.csv')
train_df = all_df[:int(len(all_df)*0.9)]
test_df = all_df[int(len(all_df)*0.9):]

#학습 데이터 전처리, 영어 특수문자 제거, 공백 제거
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['document'] = train_df['document'].apply(isAll0)
train_df['document'].replace('', np.nan, inplace=True)

#테스트 데이터 전처리
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'] = test_df['document'].apply(isAll0)
test_df['document'].replace('', np.nan, inplace=True)

#중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

#Null 제거
train_df = train_df.dropna(how = 'any')
test_df = test_df.dropna(how = 'any')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [304]:
zero = 0
one = 0
for i in all_df.values:
    if i[1] == 0:
        zero += 1
    else:
        one += 1
print(zero,one)
print(zero/one)

4441 3497
1.2699456677151844


In [305]:
print(len(train_df))
print(len(test_df))

7142
794


In [306]:
#불용어
#stopwords = {'의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', ' '}

In [307]:
train_x = []

In [308]:
#학습 데이터 형태소로 분리, 문장 시작, 끝 추가
j = 0
for i, row in train_df.iterrows():
    val = row['document']
    j += 1
    if j % 10000 == 0:
        print(i,"/","143660")

    temp = ['CLS']
    '''
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    '''
    for word in val:
        temp.append(word)
    temp.append('SEP')
    train_x.append(temp)



In [309]:
test_x = []

In [310]:
#테스트 데이터 형태소로 분리
j = 0
for i, row in test_df.iterrows():
    val = row['document']
    j += 1
    if i % 10000 == 0:
        print(j,"/","48000")
    temp = ['CLS']
    '''
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    '''
    for word in val:
        temp.append(word)
    temp.append('SEP')
    test_x.append(temp)


In [311]:
print(train_x)

[['CLS', '현', '재', ' ', '호', '텔', '주', '인', ' ', '심', '정', ' ', '아', ' ', '난', ' ', '마', '른', '하', '늘', '에', ' ', '날', '벼', '락', '맞', '고', ' ', '호', '텔', '망', '하', '게', '생', '겼', '는', '데', ' ', '누', '군', ' ', '계', '속', ' ', '추', '모', '받', '네', 'SEP'], ['CLS', '한', '국', '적', '인', ' ', '미', '인', '의', ' ', '대', '표', '적', '인', ' ', '분', '너', '무', '나', ' ', '곱', '고', '아', '름', '다', '운', '모', '습', '그', '모', '습', '뒤', '의', ' ', '슬', '픔', '을', ' ', '미', '처', ' ', '알', '지', '못', '했', '네', '요', 'ㅠ', 'SEP'], ['CLS', '못', '된', ' ', '넘', '들', '남', '의', ' ', '고', '통', '을', ' ', '즐', '겼', '던', ' ', '넘', '들', '이', '젠', ' ', '마', '땅', '한', ' ', '처', '벌', '을', ' ', '받', '아', '야', '지', '그', '래', '야', ' ', '공', '정', '한', ' ', '사', '회', '지', '심', '은', '대', '로', ' ', '거', '두', '거', '라', 'SEP'], ['CLS', '화', ' ', '어', '설', '펐', '는', '데', ' ', '화', ' ', '지', '나', '서', '부', '터', '는', ' ', '갈', '수', '록', ' ', '너', '무', ' ', '재', '밌', '던', '데', 'SEP'], ['CLS', ' ', '사', '람', ' ', '얼', '굴', ' ', '손', '톱', '으', '로

In [312]:
import json
vocab_size = 6000

#형태소를 정수로 인코딩
tokenizer = Tokenizer(vocab_size,oov_token = 'OOV')
tokenizer.fit_on_texts(train_x)
'''
with open('/content/gdrive/My Drive/GRUmodel/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
'''
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [313]:
train_y = np.array(train_df['lable'])
test_y = np.array(test_df['lable'])

In [314]:
# 제거 후 빈칸만 남은 거 또한 제거
drop_train = [index for index, sentence in enumerate(train_x) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(test_x) if len(sentence) < 1]

In [315]:
train_x = np.delete(train_x, drop_train, axis=0)
train_y = np.delete(train_y, drop_train, axis=0)
print(len(train_x))
print(len(train_y))

7142
7142


In [316]:
test_x = np.delete(test_x, drop_test, axis=0)
test_y = np.delete(test_y, drop_test, axis=0)
print(len(test_x))
print(len(test_y))

794
794


In [317]:
#패딩
pad_len = 200
train_x = pad_sequences(train_x, maxlen = pad_len,padding='post')
test_x = pad_sequences(test_x, maxlen = pad_len,padding='post')

In [318]:
#데이터 셋 간단하게
class nlp_dataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [319]:
#베이스 라인
class grubase(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grubase, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x = x[:,-1,:]
        #x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)

        x = self.dropout(x)
        x = self.mlp1(x)
        #x = self.sigmoid(x).squeeze()
        return x.squeeze()


In [320]:
# 튜닝
class grumodel(nn.Module):
    def __init__(self, embed_dim, vocab_size, hidden_dim, num_layers, batch_size, dropout):
        super(grumodel, self).__init__()
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.relu = nn.ReLU()
        self.batch_size = batch_size
        self.sigmoid = nn.Sigmoid()


        self.norm = nn.BatchNorm1d(self.batch_size)
        self.embed = nn.Embedding(self.vocab_size,self.embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.gru = nn.GRU(self.embed_dim, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,bidirectional=True,batch_first=True)
        self.gru3 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers,batch_first=True)

        self.mlp1 = nn.Linear(self.hidden_dim,self.hidden_dim//2)
        self.mlp2 = nn.Linear(self.hidden_dim//2,self.hidden_dim//4)
        self.mlp3 = nn.Linear(self.hidden_dim//4,1)

    def forward(self,x):
        x = self.embed(x)
        x = self.dropout(x)

        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        x, _ = self.gru3(x)
        #x = torch.cat((x[:,0,:],x[:,-1,:]),dim=-1)
        x = x[:,-1,:]
        
        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp2(x)
        x = self.relu(x)
        x = self.mlp3(x)
        return x.squeeze()


In [321]:
#학습
def train(model, optimizer, loss_function,train_loader,DEVICE):
    model.train()
    for batch in train_loader:
        x, y = batch
        x = x.long().to(DEVICE)
        y = y.long().to(DEVICE)
        optimizer.zero_grad()

        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        loss.backward()
        optimizer.step()

In [322]:
# 성능검증 용, F1 스코어, 정확도
def getF1(y_pred,y,threshold=0.5):
    
    yp = [1 if x > threshold else 0 for x in y_pred]

    pp = 0
    pf = 0
    fp = 0
    ff = 0
    for i in range(len(y)):
        if y[i] > threshold:
            if yp[i] > threshold: pp += 1
            else: pf += 1
        else:
            if yp[i] < threshold: ff += 1
            else: fp += 1

    precision = pp / (pp + fp + 1e-5) 
    recall = pp / (pp + ff + 1e-5)
    F1 = 2 * precision * recall / (precision + recall + 1e-5)
    acc = (pp + ff) / (len(y) + 1e-5)
    return F1, acc

In [323]:
# 검증
def evaluate(model, val_loader, loss_function, DEVICE, batch_size, threshold):
    """evaluate model"""
    model.eval()
    total_loss = 0
    total_f1 = 0
    total_acc = 0
    for batch in val_loader:
        x, y = batch
        x = x.long().to(DEVICE)
        y = y.long().to(DEVICE)
        y_pred = model(x)
        loss = loss_function(y_pred.to(DEVICE).float(), y.float())
        f1, acc = getF1(y_pred,y,threshold)
        total_f1 += f1
        total_acc += acc
        total_loss += loss.item()

    size = len(val_loader.dataset) / batch_size
    avg_loss = total_loss / size
    avg_f1 = total_f1 / size
    avg_acc = total_acc / size
    return avg_loss, avg_f1, avg_acc

In [335]:
device = torch.device("cuda")

batch_size = 256
embed_dim = 128
hidden_dim = 256
dropout = 0.3
layers = 1

model = grumodel(embed_dim,vocab_size,hidden_dim,layers,batch_size,dropout)
#modelPath = '/content/gdrive/My Drive/GRUmodel/Khaiii_gru_model.pt'
#model.load_state_dict(torch.load(modelPath))
model.to(device)
loss = nn.BCEWithLogitsLoss(pos_weight = 1 * torch.ones([1])).to(device)
lr = 0.003
threshold = 0.5

EPOCHS = 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [336]:
train_dataset = nlp_dataset(train_x,train_y)
test_dataset = nlp_dataset(test_x,test_y)

train_loader = DataLoader(train_dataset,batch_size,True,drop_last=True)
val_loader = DataLoader(test_dataset,batch_size,True,drop_last=True)

In [337]:
print(train_dataset[0])

(array([  4, 126,  76,   2, 207, 776,  56,  23,   2, 119,  46,   2,  13,
         2, 130,   2,  41, 269,  10, 309,  24,   2, 291, 670, 468, 264,
         8,   2, 207, 776, 239,  10,  33,  59, 359,   6,  28,   2, 147,
       226,   2, 190, 212,   2, 322,  77, 220,  39,   5,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

In [338]:
best_val_f1 = 0
for e in range(1, EPOCHS+1):
    train(model, optimizer, loss,train_loader,device)
    val_loss,val_f1,val_acc = evaluate(model, val_loader, loss, device,batch_size,threshold)

    print("[Epoch: %d] val loss : %1.5f    val acc :%4.3f    F1 :%4.3f" % (e, val_loss, val_acc,val_f1))

    # F1 성능 지표로 저장
    if not best_val_f1 or val_f1 > best_val_f1:
        print("Best saved")
        torch.save(model.state_dict(), '/content/gdrive/My Drive/GRUmodel/Khaiii_best_model.pt')
        best_val_f1 = val_f1

[Epoch: 1] val loss : 0.66653    val acc :0.535    F1 :0.000
Best saved
[Epoch: 2] val loss : 0.65730    val acc :0.533    F1 :0.000
Best saved
[Epoch: 3] val loss : 0.62971    val acc :0.537    F1 :0.000
Best saved
[Epoch: 4] val loss : 0.61449    val acc :0.637    F1 :0.469
Best saved
[Epoch: 5] val loss : 0.59218    val acc :0.647    F1 :0.357
[Epoch: 6] val loss : 0.61128    val acc :0.649    F1 :0.493
Best saved
[Epoch: 7] val loss : 0.56508    val acc :0.665    F1 :0.454
[Epoch: 8] val loss : 0.63694    val acc :0.659    F1 :0.486
[Epoch: 9] val loss : 0.56220    val acc :0.676    F1 :0.466
[Epoch: 10] val loss : 0.60613    val acc :0.678    F1 :0.497
Best saved
[Epoch: 11] val loss : 0.60281    val acc :0.680    F1 :0.467
[Epoch: 12] val loss : 0.61231    val acc :0.675    F1 :0.465
[Epoch: 13] val loss : 0.60109    val acc :0.656    F1 :0.384
[Epoch: 14] val loss : 0.68215    val acc :0.654    F1 :0.457
[Epoch: 15] val loss : 0.76717    val acc :0.670    F1 :0.476
[Epoch: 16] v

In [339]:
model.eval()
def predict(original_sentence):
    sentence = ['CLS']
    '''
    for word in api.analyze(original_sentence):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                    sentence.append(morph.lex)
                    '''
    for word in original_sentence:
        sentence.append(word)

    sentence.append('SEP')
    x = tokenizer.texts_to_sequences([sentence])
    x = pad_sequences(x, maxlen = pad_len,padding='post') # 패딩
    x = torch.tensor(x).to(device).long()
    
    y = model(x)
    if(y > 0):
        return 'P'
    else:
        return 'B'

In [340]:
print(predict("개소리 오지네"))
print(predict("안녕?"))
print(predict("아니 씨발"))
print(predict("선동 ㄴㄴ"))
print(predict("ㅁㅊ ㅅㅂ"))
print(predict("ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ"))
print(predict("존나 웃기네"))
print(predict("안녕하세요"))
print(predict("개소리를 길게도 쓰셨네요"))
print(predict("나가 뒤지세요"))

B
P
B
B
B
B
B
P
B
P


In [341]:
'''
import json
tokenizer_json = tokenizer.to_json()
with open('/content/gdrive/My Drive/GRUmodel/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
'''

"\nimport json\ntokenizer_json = tokenizer.to_json()\nwith open('/content/gdrive/My Drive/GRUmodel/tokenizer.json', 'w', encoding='utf-8') as f:\n    f.write(json.dumps(tokenizer_json, ensure_ascii=False))\n"